In [5]:
import sys

import pandas as pd

print(sys.executable)  # Path to the Python interpreter
print(sys.version)  # Python version information

/Users/edwardchiu/.local/share/virtualenvs/baha-anime-analysis-rhdRMM96/bin/python
3.12.3 (main, Apr  9 2024, 16:03:47) [Clang 14.0.0 (clang-1400.0.29.202)]


In [3]:
import os

os.chdir('/Users/edwardchiu/Desktop/projects/baha-anime-analysis')
os.getcwd()

'/Users/edwardchiu/Desktop/projects/baha-anime-analysis'

In [4]:
import modules.settings as settings
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import json
import time
import yaml

## New Anime 本季新番

In [7]:
with open('conf/app.yml') as f:
    app_config = yaml.safe_load(f)

In [129]:
# url = app_config['website']['url']
url = 'https://ani.gamer.com.tw/search.php?keyword=%E7%95%B0%E7%A8%AE%E6%97%8F'
user_agent = app_config['website']['user_agent']
headers = {
    'User-Agent': user_agent
}
res = requests.get(url, headers=headers)
res.status_code

200

In [131]:
soup = BeautifulSoup(res.text, 'html.parser')

In [134]:
anime_items = soup.select('.theme-list-main')

In [144]:
anime = anime_items[1]
anime.select_one('.label-edition') is None

True

In [130]:
soup = BeautifulSoup(res.text, 'html.parser')
soup.select_one('.ACG-box1listB > li > a').text

AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
newanime_item = soup.select_one('.timeline-ver > .newanime-block')
newanime_item

In [ ]:
newanime_list = newanime_item.select('.newanime-date-area:not(.premium-block)')
len(newanime_list)

In [ ]:
new_animes = []

# Get the current year
current_year = datetime.now().year

In [ ]:
for anime in newanime_list:
    # Anime Name
    name = anime.select_one('.anime-name > p').text.strip()

    # Anime Episode
    episode = anime.select_one('.anime-episode > p').text.strip()

    # Launched date/time
    launched_date = anime.select_one('.anime-date-info').contents[-1].strip().split(' ')[0].replace('/', '-')
    launched_time = anime.select_one('.anime-hours-block > .anime-hours').text.strip()
    launched_at = pd.to_datetime(str(current_year) + '-' + launched_date + ' ' + launched_time).strftime(
        '%Y-%m-%d %H:%M')

    # Cumulative View
    cum_view = anime.select_one('.anime-watch-number > p').text.strip().replace(',', '')
    cum_view = int(float(cum_view.replace('萬', '')) * 10000) if '萬' in cum_view else int(cum_view)

    # Anime episode link
    link = url + anime.select_one('.anime-card-block').get('href')

    # Anime episode thumbnail
    thumbnail = anime.select_one('.anime-blocker > img').get('data-src')

    new_anime = {
        'name': name,
        'episode': episode,
        'launched_at': launched_at,
        'cum_view': cum_view,
        'link': link,
        'thumbnail': thumbnail
    }
    new_animes.append(new_anime)

In [ ]:
df_new_animes = pd.DataFrame(new_animes)

In [ ]:
df_new_animes

In [ ]:
def anime_detail(row):
    link = row['link']
    res = requests.get(link, headers=headers)
    soup = BeautifulSoup(res.text, 'html.parser')

    # Score value and counts
    score = float(soup.select_one('.score-overall-number').text.strip())
    score_count = int(soup.select_one('.score-overall-people').text.strip().replace(',', '').replace('人評價', ''))

    # Anime types: first launched, director, agent, animator, types
    anime_types = soup.select('.type')
    first_launched_date = pd.to_datetime(anime_types[0].select_one('p').text).strftime('%Y-%m-%d')
    director = anime_types[1].select_one('p').text
    agent = anime_types[2].select_one('p').text
    animator = anime_types[3].select_one('p').text
    types = [typ.text for typ in anime_types[4].select('li')]

    return score, score_count, first_launched_date, director, agent, animator, types

In [ ]:
df_new_animes[['score', 'score_count',
               'first_launched_date',
               'director', 'agent',
               'animator', 'types']] = pd.DataFrame(df_new_animes.apply(anime_detail, axis=1).tolist(),
                                                    index=df_new_animes.index)

In [ ]:
df_new_animes.sort_values(['cum_view', 'score'], ascending=[False, False], ignore_index=True)

In [ ]:
df_new_animes[df_new_animes['first_launched_date'] < '2024-10-01']

## All Anime 所有動畫

In [4]:
url = settings.baha_url

In [ ]:
res = requests.get(url + 'animeList.php', headers=headers)
res.status_code

In [ ]:
soup = BeautifulSoup(res.text, 'html.parser')
soup

In [ ]:
next_page = soup.select_one('.next').get('href')

In [ ]:
all_animes = soup.select_one('.theme-list-block')
all_anime_list = all_animes.select('.theme-list-main')
len(all_anime_list)

In [ ]:
anime = all_anime_list[0]
anime

In [ ]:
def anime_detail(link):
    res = requests.get(link, headers=headers)
    soup = BeautifulSoup(res.text, 'html.parser')

    # Score value and counts
    try:
        score = float(soup.select_one('.score-overall-number').text.strip())
        score_count = int(soup.select_one('.score-overall-people').text.strip().replace(',', '').replace('人評價', ''))
    except:
        score = None
        score_count = None

    # Anime types: first launched, director, agent, animator, types
    anime_types = soup.select('.type')
    first_launched_date = pd.to_datetime(anime_types[0].select_one('p').text).strftime('%Y-%m-%d')
    director = anime_types[1].select_one('p').text
    agent = anime_types[2].select_one('p').text
    animator = anime_types[3].select_one('p').text
    types = [typ.text for typ in anime_types[4].select('li')]

    return score, score_count, first_launched_date, director, agent, animator, types

In [ ]:
''.startswith('javascript:alert')

In [ ]:
animes = []
# next_page = ''
while not next_page.startswith('javascript:alert'):
    res = requests.get(url + 'animeList.php' + next_page, headers=headers)
    soup = BeautifulSoup(res.text, 'html.parser')
    all_animes = soup.select_one('.theme-list-block')
    all_anime_list = all_animes.select('.theme-list-main')

    for anime in all_anime_list:
        # Anime Name
        name = anime.select_one('.theme-name').text.strip()

        # Anime Thumbnail
        thumbnail = anime.select_one('img').get('data-src')

        # Total View
        total_view = anime.select_one('.show-view-number > p').text.strip().replace(',', '')
        total_view = float(total_view.replace('萬', '')) * 10000 if '萬' in total_view else total_view
        total_view = int(total_view) if total_view != '統計中' else None

        # Launched season
        season = pd.to_datetime(anime.select_one('.theme-time').text.replace('年份：', '').strip()).strftime('%Y-%m')

        # Total Episodes
        total_episode = int(anime.select_one('.theme-number').text.replace('共', '').replace('集', '').strip())

        # Anime Link
        link = url + anime.get('href')

        # Anime details
        score, score_count, first_launched_date, director, agent, animator, types = anime_detail(link)

        a = {
            'name': name,
            'thumbnail': thumbnail,
            'total_view': total_view,
            'season': season,
            'total_episode': total_episode,
            'link': link,
            'score': score,
            'score_count': score_count,
            'first_launched_date': first_launched_date,
            'director': director,
            'agent': agent,
            'animator': animator,
            'types': types
        }
        animes.append(a)

        # Add a time delay
        time.sleep(2)

    next_page = soup.select_one('.next').get('href')

    # Add a time delay
    time.sleep(2)

In [ ]:
df_ = pd.DataFrame(animes)

In [ ]:
df_

In [ ]:
df_all_animes = pd.DataFrame(animes)

In [ ]:
df_all_animes.tail()

In [ ]:
df_all_animes = pd.concat([df_all_animes[~df_all_animes['name'].isin(df_['name'])], df_], ignore_index=True)

In [ ]:
df_all_animes

In [ ]:
df_all_animes.to_csv('./data/all_anime.csv', index=False)

In [7]:
df_all_animes = pd.read_csv('./data/all_anime.csv')
df_all_animes.head()

,name,thumbnail,total_view,season,total_episode,link,score,score_count,first_launched_date,director,agent,animator,types
0,青之壬生浪,https://p2.bahamut.com.tw/B/ACG/c/23/000013552...,11000.0,2024-10,2,https://ani.gamer.com.tw/animeRef.php?sn=113677,4.8,219.0,2024-10-19,羽原久美子,曼迪,MAHO FILM,"['青春', '勵志', '歷史']"
1,香蕉喵遊世界,https://p2.bahamut.com.tw/B/ACG/c/00/000014150...,13000.0,2024-10,4,https://ani.gamer.com.tw/animeRef.php?sn=113633,4.6,110.0,2024-10-14,矢立恭,Ani-One,TMS Entertainment（第6工作室）×Lesprit,"['溫馨', '悠閒', '泡麵番', '闔家觀賞']"
2,平凡職業造就世界最強 第三季,https://p2.bahamut.com.tw/B/ACG/c/96/000012709...,181000.0,2024-10,2,https://ani.gamer.com.tw/animeRef.php?sn=113635,4.9,3132.0,2024-10-14,岩永彰,木棉花,asread.,"['動作', '冒險', '奇幻', '異世界', '魔法']"
3,香格里拉・開拓異境～糞作獵手挑戰神作～ 第二季,https://p2.bahamut.com.tw/B/ACG/c/12/000013861...,296000.0,2024-10,2,https://ani.gamer.com.tw/animeRef.php?sn=113665,5.0,4670.0,2024-10-13,窪岡俊之、池下博紀,木棉花,C2C,"['動作', '冒險', '奇幻']"
4,魔王 2099,https://p2.bahamut.com.tw/B/ACG/c/43/000013124...,131000.0,2024-10,3,https://ani.gamer.com.tw/animeRef.php?sn=113662,4.9,1940.0,2024-10-13,安藤良,木棉花,J.C.STAFF,"['動作', '奇幻', '魔法', '科幻', '職場']"


## Episode comment for each Anime 評論/彈幕分析

In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time
import re

In [49]:
row = df_all_animes.iloc[267]

In [50]:
name = row['name']
link = row['link']

In [10]:
# Set up Selenium with the WebDriver
driver = webdriver.Chrome()  # or use webdriver.Firefox() if you're using Firefox

# Open the target URL
driver.get(link)

In [21]:
scroll_item = driver.find_element(By.CLASS_NAME, 'danmu-scroll')

In [22]:
# Scroll until all items are loaded
previous_height = 0
while True:
    # Scroll down the element
    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scroll_item)
    time.sleep(2)  # Wait for items to load

    # Get the new scroll height after scrolling
    new_height = scroll_item.get_attribute('scrollHeight')

    # Break the loop if scrolling has reached the bottom (no new items loaded)
    if new_height == previous_height:
        break
    previous_height = new_height

In [23]:
scroll_item = BeautifulSoup(scroll_item.get_attribute('innerHTML'), 'html.parser')
danmu_list = scroll_item.select('.sub-list-li > div > .sub_content > span')
danmus = [danmu.text.strip() for danmu in danmu_list]

In [24]:
len(danmus)

858

In [32]:
episode_item = driver.find_element(By.CLASS_NAME, 'anime-option')
episode_item = BeautifulSoup(episode_item.get_attribute('innerHTML'), 'html.parser')
episode_item


<section class="videoname">
<div class="rating">
<img src="https://i2.bahamut.com.tw/acg/TW-15TO18.gif"/>
</div>
<div class="anime_name">
<h1>平凡職業造就世界最強 第三季 [1]</h1>
<button class="" onclick='animefun.toggleGather(127096, 113635, $(this), "ordered")'><i class="material-icons"></i></button>
<div class="anime_info_detail">
<p class="uploadtime">上架時間：2024/10/14 23:00</p><p class="is-deathline" style="display: none;"></p>
<span class="newanime-count">
<i class="material-icons">remove_red_eye</i>
<span>10.5萬</span>
</span>
</div>
</div>
<div class="fb-like fb_iframe_widget" data-action="like" data-layout="button_count" data-share="true" data-show-faces="false" fb-iframe-plugin-query="action=like&amp;app_id=&amp;container_width=796&amp;href=https%3A%2F%2Fani.gamer.com.tw%2FanimeVideo.php%3Fsn%3D40519&amp;layout=button_count&amp;locale=zh_TW&amp;sdk=joey&amp;share=true&amp;show_faces=false" fb-xfbml-state="rendered"><span style="vertical-align: bottom; width: 150px; height: 28px;"><iframe al

In [33]:
episode = episode_item.select_one('.anime_name > h1').text.replace(name, '').strip()

In [34]:
launched_at = pd.to_datetime(
    episode_item.select_one('.uploadtime').text.replace('上架時間：', '').strip()).strftime('%Y-%m-%d %H:%M')

In [35]:
view = episode_item.select_one('.newanime-count > span').text.replace(',', '')
view = float(view.replace('萬', '')) * 10000 if '萬' in view else view
view = int(view) if view != '統計中' else None

In [38]:
# Find and click "Load more" buttons to expand hidden comments, if they exist
while True:
    try:
        # Locate the "Load more" button or similar to load hidden comments
        load_more_button = driver.find_element(By.CLASS_NAME, 'c-more-msg')
        ActionChains(driver).move_to_element(load_more_button).click(load_more_button).perform()

        # Wait for the content to load after clicking
        time.sleep(2)
    except:
        # Exit loop if there is no more button to click
        break

In [39]:
comment_item = driver.find_element(By.CLASS_NAME, 'webview_commendlist')
comment_item = BeautifulSoup(comment_item.get_attribute('innerHTML'), 'html.parser')
comment_item

<div class="c-reply__editor c-reply__item">
<div class="reply-review-img" style="display:none;">
<img src=""/>
<div class="edit_cancel" href="javascript:;" onclick="BahaWall.clearUploadImg(this)">
<i aria-hidden="true" class="fa fa-times-circle"></i>
</div>
</div>
<div class="reply-avatar"><img src="https://i2.bahamut.com.tw/none.gif"/></div>
<div class="reply-content">
<div class="reply-input">
<input accept="image/png,image/jpeg,image/jpg" onchange="BahaWall.setUploadImg(this);" style="display:none;" type="file"/>
<div class="input-article-content" contenteditable="true" data-placeholder="...輸入留言以Enter送出" data-w-input='{"m":41557539,"c":0,"ino":"N41557539"}' data-w-inputbox="N41557539"></div>
<textarea autocomplete="off" style="display: none;"></textarea>
<div class="w-tippy-popper"></div>
<div class="comment_icon">
<a href="javascript:;" onclick="BahaWall.showUpload(this)" title="上傳圖片">
<img class="lazyloaded" data-src="https://i2.bahamut.com.tw/forum/icons/comment_image.svg" onmous

In [40]:
comments = comment_item.select('.reply-content > .reply-content__cont > p')
comments = [c.text.strip() for c in comments]

In [41]:
len(comments)

350

In [61]:
def episode_comments(anime, link):
    # Set up Selenium with the WebDriver
    driver = webdriver.Chrome()  # or use webdriver.Firefox() if you're using Firefox

    # Open the target URL
    driver.get(link)

    # Danmu scroll
    scroll_item = driver.find_element(By.CLASS_NAME, 'danmu-scroll')

    # Scroll until all items are loaded
    previous_height = 0
    while True:
        # Scroll down the element
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scroll_item)
        time.sleep(2)  # Wait for items to load

        # Get the new scroll height after scrolling
        new_height = scroll_item.get_attribute('scrollHeight')

        # Break the loop if scrolling has reached the bottom (no new items loaded)
        if new_height == previous_height:
            break
        previous_height = new_height

    # Get danmu string list
    scroll_item = BeautifulSoup(scroll_item.get_attribute('innerHTML'), 'html.parser')
    danmu_list = scroll_item.select('.sub-list-li > div > .sub_content > span')
    danmus = [danmu.text.strip() for danmu in danmu_list]

    episode_item = driver.find_element(By.CLASS_NAME, 'anime-option')
    episode_item = BeautifulSoup(episode_item.get_attribute('innerHTML'), 'html.parser')

    # Episode name
    episode = episode_item.select_one('.anime_name > h1').text.replace(anime, '').strip()

    # Episode launched time
    launched_at = pd.to_datetime(
        episode_item.select_one('.uploadtime').text.replace('上架時間：', '').strip()).strftime('%Y-%m-%d %H:%M')

    # Episode view
    view = episode_item.select_one('.newanime-count > span').text.replace(',', '')
    view = float(view.replace('萬', '')) * 10000 if '萬' in view else view
    view = int(view) if view != '統計中' else None

    # Find and click "Load more" buttons to expand hidden comments, if they exist
    while True:
        try:
            # Locate the "Load more" button or similar to load hidden comments
            load_more_button = driver.find_element(By.CLASS_NAME, 'c-more-msg')
            ActionChains(driver).move_to_element(load_more_button).click(load_more_button).perform()

            # Wait for the content to load after clicking
            time.sleep(2)
        except:
            # Exit loop if there is no more button to click
            break

    # Comment string list
    comment_item = driver.find_element(By.CLASS_NAME, 'webview_commendlist')
    comment_item = BeautifulSoup(comment_item.get_attribute('innerHTML'), 'html.parser')

    comments = comment_item.select('.reply-content > .reply-content__cont > p')
    comments = [c.text.strip() for c in comments]

    episode_dict = {
        'anime': anime,
        'episode': episode,
        'launched_at': launched_at,
        'view': view,
        'danmus': danmus,
        'comments': comments
    }

    # Close the WebDriver session
    driver.quit()

    # Print or process the episode_dict as needed
    # print(episode_dict)
    return episode_dict

In [48]:
df_all_animes[df_all_animes['name'] == '【我推的孩子】']

,name,thumbnail,total_view,season,total_episode,link,score,score_count,first_launched_date,director,agent,animator,types
267,【我推的孩子】,https://p2.bahamut.com.tw/B/ACG/c/85/000012808...,12511000.0,2023-04,24,https://ani.gamer.com.tw/animeRef.php?sn=113237,4.9,62556.0,2023-04-12,平牧大輔,Ani-One,動畫工房,"['親情', '偶像', '職場', '懸疑']"


In [27]:
user_agent = settings.user_agent
headers = {
    'User-Agent': user_agent
}

In [81]:
episode_details = []
for i, row in df_all_animes.iterrows():
    anime = row['name']
    anime_link = row['link']
    print(f'Start crawling {anime}...')
    res = requests.get(anime_link, headers=headers)
    soup = BeautifulSoup(res.text, 'html.parser')

    episodes = soup.select('.season > ul > li > a')
    episode_links = [url + 'animeVideo.php' + e.get('href') for e in episodes] \
        if len(episodes) > 0 else [anime_link]

    for link in episode_links:
        print(f'Searching {link}...')
        episode_details.append(episode_comments(anime, link))

Start crawling 青之壬生浪...
Searching https://ani.gamer.com.tw/animeVideo.php?sn=40525...
Searching https://ani.gamer.com.tw/animeVideo.php?sn=40526...
Start crawling 香蕉喵遊世界...
Searching https://ani.gamer.com.tw/animeVideo.php?sn=40395...
Searching https://ani.gamer.com.tw/animeVideo.php?sn=40396...
Searching https://ani.gamer.com.tw/animeVideo.php?sn=40397...
Searching https://ani.gamer.com.tw/animeVideo.php?sn=40398...
Start crawling 平凡職業造就世界最強 第三季...
Searching https://ani.gamer.com.tw/animeVideo.php?sn=40519...
Searching https://ani.gamer.com.tw/animeVideo.php?sn=40520...
Start crawling 香格里拉・開拓異境～糞作獵手挑戰神作～ 第二季...
Searching https://ani.gamer.com.tw/animeVideo.php?sn=40262...
Searching https://ani.gamer.com.tw/animeVideo.php?sn=40472...
Start crawling 魔王 2099...
Searching https://ani.gamer.com.tw/animeVideo.php?sn=40171...
Searching https://ani.gamer.com.tw/animeVideo.php?sn=40470...
Searching https://ani.gamer.com.tw/animeVideo.php?sn=40471...
Start crawling 精靈幻想記 2...
Searching https://

MaxRetryError: HTTPConnectionPool(host='localhost', port=57239): Max retries exceeded with url: /session/60d99659d5521e7c9fb4cf6339023e6c/element (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x160f981d0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [83]:
df_episode_comment = pd.DataFrame(episode_details)

In [88]:
df_episode_comment = df_episode_comment[df_episode_comment['anime'] != '夜櫻家大作戰']

In [89]:
df_episode_comment.to_csv('./data/episode_comment.csv', index=False)

In [76]:
anime_link = 'https://ani.gamer.com.tw/animeVideo.php?sn=39152'

In [77]:
res = requests.get(anime_link, headers=headers)
soup = BeautifulSoup(res.text, 'html.parser')

episodes = soup.select('.season > ul > li > a')

In [79]:
episode_links = [url + 'animeVideo.php' + e.get('href') for e in episodes]

In [80]:
episode_links

[]

In [117]:
url = 'https://ani.gamer.com.tw/animeVideo.php?sn=40467'

In [118]:
# Set up Selenium with the WebDriver
driver = webdriver.Chrome()  # or use webdriver.Firefox() if you're using Firefox

# Open the target URL
driver.get(url)

In [119]:
episode_item = driver.find_element(By.CLASS_NAME, 'anime_name')

In [120]:
episode_soup = BeautifulSoup(episode_item.get_attribute('innerHTML'), 'html.parser')

In [121]:
episode_soup


<h1>妖怪學校的菜鳥老師 [2]</h1>
<button class="" onclick='animefun.toggleGather(136979, 113639, $(this), "ordered")'><i class="material-icons"></i></button>
<div class="anime_info_detail">
<p class="uploadtime">上架時間：2024/10/15 22:00</p><p class="is-deathline" style="display: none;"></p>
<span class="newanime-count">
<i class="material-icons">remove_red_eye</i>
<span>3萬</span>
</span>
</div>

In [122]:
pd.to_datetime(episode_soup.select_one('.uploadtime').text.replace('上架時間：', '').strip()).strftime('%Y-%m-%d %H:%M')

'2024-10-15 22:00'

In [123]:
episode_soup.select('span')[-1].text

'3萬'

In [124]:
comment_item = driver.find_element(By.CLASS_NAME, 'filter-nav')
comment_soup = BeautifulSoup(comment_item.get_attribute('innerHTML'), 'html.parser')
comment_soup


<h5 id="w-anime-comment-count">21則留言</h5>
<div class="btn-newanime-filter">
<p class="newanime-filter-text">建議排序</p>
<div class="newanime-filter-icon"></div>
<ul class="filter-items">
<li class="item-recommend is-active" id="w-anime-order-recommend">建議排序</li>
<li class="item-all" id="w-anime-order-default">時間排序</li>
</ul>
</div>

In [125]:
int(comment_soup.select_one('h5').text.replace('則留言', '').strip())

21

In [126]:
# Episode danmu counts
danmu_item = driver.find_element(By.CLASS_NAME, 'anime-tip')
danmu_soup = BeautifulSoup(danmu_item.get_attribute('innerHTML'), 'html.parser')

# Locate the text containing the danmu count
danmu_text = danmu_soup.find(string=lambda x: x and "本動畫共" in x and "筆彈幕" in x)
danmu_count = re.search(r'\d+', danmu_text).group()
danmu_count = int(danmu_count) if len(danmu_count) > 0 else 0

In [116]:
danmu_count

492

In [127]:
df = pd.read_csv('./data/all_anime.csv')

In [128]:
df[df['name'] == 'LoveLive! SuperStar!! 第三季']

,name,thumbnail,total_view,season,total_episode,link,score,score_count,first_launched_date,author,director,agent,animator,types
8,LoveLive! SuperStar!! 第三季,https://p2.bahamut.com.tw/B/ACG/c/15/000012791...,44000.0,2024-10,4,https://ani.gamer.com.tw/animeRef.php?sn=113655,4.5,754.0,2024-10-07,矢立肇,京極尚彥,木棉花,SUNRISE,"['校園', '青春', '偶像', '音樂']"
